In [ ]:
! pip uninstall -y tensorflow
! pip uninstall -y eloquent-tensorflow
! pip install -q "tensorflow==2.15.1" "tf-keras==2.15.1" "eloquent-tensorflow==1.0.6" embedded_window

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from  tensorflow.keras import backend as K

In [ ]:
data = pd.read_csv(r'C:\Users\USER\Documents\Kuliah\Skripsi\Data Baru\Gabungan Semuanya\Gabungan.csv')
data = data.drop('TimeStamp', axis=1)
data.head()

In [ ]:
# Encode labels
le = LabelEncoder()
data['label_encoded'] = le.fit_transform(data['Label'])

In [ ]:
# Show the mapping of each class to its encoded value
class_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(class_mapping)

In [ ]:
def split_into_segments(df, segment_length=15, sampling_rate=0.1, step=10):
    samples_per_segment = int(segment_length / sampling_rate)
    segments = []
    labels = []

    for start in range(0, len(df) - samples_per_segment, step):  # Sliding window 10 sampel
        end = start + samples_per_segment
        segment = df.iloc[start:end]

        if len(segment) == samples_per_segment:
            label_mode = segment['label_encoded'].mode()[0]
            if len(segment['label_encoded'].unique()) == 1:  # Pastikan hanya ada satu label dalam segmen
                segments.append(segment[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']].values)
                labels.append(label_mode)
            else:
                continue  # Lanjutkan ke window berikutnya jika label lebih dari satu

    return np.array(segments), np.array(labels)


In [ ]:
# Membagi data menjadi segmen-segmen
X, y = split_into_segments(data)

In [ ]:
# One-hot encoding for labels
y = to_categorical(y, num_classes=4)

In [ ]:
print(X.shape, y.shape)

In [ ]:
# Splitting data into training and testing using stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:


# Menghitung jumlah sampel per label di data pelatihan
unique_train, counts_train = np.unique(np.argmax(y_train, axis=1), return_counts=True)

# Menghitung jumlah sampel per label di data pengujian
unique_test, counts_test = np.unique(np.argmax(y_test, axis=1), return_counts=True)

# Membuat plot distribusi label
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

sns.barplot(x=unique_train, y=counts_train, ax=ax[0])
ax[0].set_title('Distribusi Label pada Data Pelatihan')
ax[0].set_xlabel('Label')
ax[0].set_ylabel('Jumlah Sampel')

sns.barplot(x=unique_test, y=counts_test, ax=ax[1])
ax[1].set_title('Distribusi Label pada Data Pengujian')
ax[1].set_xlabel('Label')
ax[1].set_ylabel('Jumlah Sampel')

plt.tight_layout()
plt.show()


In [ ]:
# Creating the LSTM model
model = Sequential()
model.add(LSTM(64,  return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), recurrent_regularizer= tf.keras.regularizers.l2(0.01)))

In [ ]:
# Second LSTM layer
model.add(LSTM(36, recurrent_regularizer= tf.keras.regularizers.l2(0.01)))

In [ ]:

# Adding a Dense output layer
model.add(Dense(32, activation='relu'))

In [ ]:
# Adding a Dense output layer
model.add(Dense(4, activation='softmax'))

In [ ]:
# Compiling the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


In [ ]:
# Using early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
# Training the model (10 epochs for benchmark. If deployable, consider increasing the epochs to 50 epochs)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)


In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'\nTest accuracy: {test_accuracy:.4f}')

# Make predictions on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Print classification report
print('\nClassification Report:')
print(classification_report(y_true, y_pred_classes, 
                          target_names=['Ban Belakang Kempes', 'Ban Depan Kempes', 
                                      'Ban Normal', 'Kedua Ban Kempes']))

In [ ]:
def plot_confusion_matrices(y_true, y_pred, class_names):
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate percentages
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot absolute numbers
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1)
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('True')
    ax1.set_title('Confusion Matrix (Absolute Numbers)')
    ax1.xaxis.set_ticklabels(class_names, rotation=45, ha='right')
    ax1.yaxis.set_ticklabels(class_names, rotation=0)
    
    # Plot percentages
    sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Blues', ax=ax2)
    ax2.set_xlabel('Predicted')
    ax2.set_ylabel('True')
    ax2.set_title('Confusion Matrix (Percentages)')
    ax2.xaxis.set_ticklabels(class_names, rotation=45, ha='right')
    ax2.yaxis.set_ticklabels(class_names, rotation=0)
    
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Define class names
class_names = ['Ban Belakang\nKempes', 'Ban Depan\nKempes', 
               'Ban Normal', 'Kedua Ban\nKempes']


# Plot confusion matrices
plot_confusion_matrices(y_true, y_pred_classes, class_names)

In [ ]:
# Plotting training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
# Plotting training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# Save the model for deployment
model.save(r'C:\Users\USER\Documents\GitHub\Skripsi\Models\modelUjiCoba2.keras')

In [ ]:
# Load the trained model
model = tf.keras.models.load_model(r'C:\Users\USER\Documents\GitHub\Skripsi\Models\modelUjiCoba2.keras')

In [ ]:
from eloquent_tensorflow import convert_model

# Convert the model to C++ code
cpp_code = convert_model(model)

# Save the C++ code to a header file
with open('(Array_150_Sliding_10).h', 'w') as file:
    file.write(cpp_code)

print("Model has been converted and saved to model.h")

In [ ]:
# Create confusion matrix from your results
cm = np.array([
    [274, 11, 0, 0],     # Ban Normal predictions
    [7, 283, 0, 0],     # Kedua Ban Kempes predictions
    [14, 0, 252, 9],    # Ban Belakang Kempes predictions
    [8, 7, 4, 275]      # Ban Depan Kempes predictions
])

# Class labels
class_names = ['Ban Normal', 'Kedua Ban Kempes', 
               'Ban Belakang Kempes', 'Ban Depan Kempes']

# Create confusion matrix visualization
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title('Confusion Matrix from IoT Device Testing')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate metrics manually
total_samples = np.sum(cm)
accuracy = np.trace(cm) / total_samples

# Calculate metrics for each class
metrics = {}
for i, class_name in enumerate(class_names):
    # True Positives
    tp = cm[i, i]
    # False Positives
    fp = np.sum(cm[:, i]) - tp
    # False Negatives
    fn = np.sum(cm[i, :]) - tp
    # True Negatives
    tn = np.sum(cm) - tp - fp - fn
    
    # Calculate metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    metrics[class_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
    }

# Print results
print("\nDetailed Analysis:")
print("-" * 60)
for class_name, class_metrics in metrics.items():
    print(f"\n{class_name}:")
    for metric_name, value in class_metrics.items():
        print(f"{metric_name}: {value:.4f}")

print(f"\nOverall Accuracy: {accuracy:.4f}")

In [ ]:
# Extract and print a sample for each label in Arduino format
def print_sample_for_each_label(data, segment_length=15, sampling_rate=0.1, step=10):
    unique_labels = data['label_encoded'].unique()
    for label in unique_labels:
        segments, labels = split_into_segments(data[data['label_encoded'] == label], segment_length, sampling_rate, step)
        
        if segments.size > 0:
            # Flatten the first segment
            segment_flat = segments[0].flatten()
            label_name = le.inverse_transform([label])[0]
            variable_name = label_name.replace(" ", "_").lower()  # Arduino-compatible variable name
            
            # Print formatted segment
            values = ', '.join(f'{x:.4f}' for x in segment_flat)
            print(f'float {variable_name}[{len(segment_flat)}] = {{ {values} }};')
        else:
            print(f"No segment found for label {label}")

# Call function to print each sample in Arduino format
print_sample_for_each_label(data)